In [1]:
import requests
import urllib
import json
import datetime
import time

# 1. 토큰 발급받기

https://developer.riotgames.com/

에 로그인 + 가입으로 개발자 api key를 발급받으세요.

In [2]:
my_key =  #expires on 2022.02.05 22:40 (PT = UTC-8 = KST - 17) # 우리나라보다 17시간 느린시간, 24시간 지원

# 2. Document 확인하기

- https://developer.riotgames.com/docs/portal
- - https://developer.riotgames.com/docs/lol
- 참고할 만한 내용

### 1. 호출 제한
- 1초에 20개의 호출만 가능하다.
- 2분에는 100개가 최대이다.
- 따라서, time.sleep() 설정이 필요해 보인다.

### 2. response code
- 200 : 정상 수집
- 4XX : 부적합한 request
    - 400 : 요청사항 중 안전달한게 있거나 틀린 내용이 있음
    - 401 : 주로 API Key를 전달하지 않아 생김
    - 403 : 금지된 error, 주로 차단되거나 없는 API, 없는 경로
    - 404 : Not Found : 찾을 수 없다. 주로 없는 유저 ID 등
    - 415 : 지원하지 않는 media
    - 429  호출 횟수 초과
- 4XX : 서버 에러
    - 500 : 내부 서버 에러
    - 503 : 서비스 불가
    
### 그 외
- 한국 호출 서버가 따로 있어 빠를듯 싶네요

# 3. 데이터 요청하기

## 1) 헤더 설정

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": my_key
}

## 2) 유저 정보 불러오기

- refer : https://developer.riotgames.com/apis#summoner-v4
- 유저(롤에서는 summoner, 소환사라고 부른다)의 정보를 불러오고 싶습니다.
- API로 활용할 때는, 유저 정보를 객채화한 DTO 형태로 사용해야 합니다.
    - Data Transfer Obejct

- 암호화된 정보는 다음의 5가지를 통해 얻을 수 있습니다.
    - (1) 계정 ID를 이용
    - (2) 계정 이름을 이용
    - (3) PUUID (잘 모르겠다..)
    - (4) 암호화된 소환사 ID를 이용
    - (5) 자기 자신

이번 예시에서는 롤 프로게이머 빛상혁님의 "Hide on bush"를 활용해보겠습니다 >_<
- Summoner의 계정 이름이므로, 2번을 사용합니다.

### 2번 방법의 요구사항

- user_name : url에 추가합니다.

- headers
    - User-Agent : https://www.whatismybrowser.com/detect/what-is-my-user-agent 에서 확인 가능
    - Accept-Language : 'ko-KR,ko;q=0.9' # 한국어
    - Accept-Charset: "application/x-www-form-urlencoded; charset=UTF-8", # 문자 형태는 UTF-8로 인코딩 한 값
    - Origin : 데이터 출처
    - X-Riot-Token : 개인 api key

In [4]:
user_name = 'Hide on bush'

user_name = user_name.replace(' ','%20')
request_url = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{user_name}'

In [5]:
today = datetime.datetime.today()

year = today.year
month = today.month
day = today.day
hour = today.hour

- 방법1

In [6]:
res = urllib.request.Request(request_url)

for key,value in headers.items() : 
    res.add_header(key, value)
response = urllib.request.urlopen(res)
result = response.readlines()

result = json.loads(result[0].decode('utf-8'))

print('Status Code : ',response.getcode())

Status Code :  200


- 방법2

In [7]:
response = requests.get(request_url, headers = headers)
result = response.json()
print('Status Code : ',response)

Status Code :  <Response [200]>


- 정말 잘 불러와졌는지 Test

In [8]:
user_puuid = result.get('puuid')
user_name = result.get('name')
user_level = result.get('summonerLevel')

print(f'{year}년 {month}월 {day}일 {hour}시 기준\n{user_name}님은 {user_level}레벨입니다.')

2022년 2월 5일 17시 기준
Hide on bush님은 534레벨입니다.


## 3) 유저 정보를 바탕으로 과거 매치 검색하기

- refer : https://developer.riotgames.com/apis#match-v5
- 우리는 위의 단계에서 빛상혁님의 "user_puuid"를 획득할 수 있었다.
- 이를 이용해서 정보를 수집할 것이다.
    - 기본 url : /lol/match/v5/matches/by-puuid/{puuid}/ids
- 전적 검색 기능 
    - puuid 기준
    - matchid 기준
    - matchId에 timeline 기준


### 요구 파라미터
- 필수
    - puuid
- 선택
    - 시작 시간 : Epoch Timestep형태로 불러오기. 
        - 이는 "유닉스 시간"이라는 것으로, 하나의 시간 개념이다.
        - https://www.epochconverter.com/ 에서 확인하고, 바꿀 수 있다.
        - mktime이라는 module로도 가능
        - 이번 실습에서는 2022년 1월1일부터를 기준으로 한다.
        - 1640998800
    - 끝 시간
        - 마찬가지로, Epoch timestep 형태로, 2022년 1월 31일을 기준으로 한다.
        - 1643641199
    - type 
        - match type
        - 다양한 매치 형태가 있으며, 이 중 어떤 타입을 볼것인지를 의미
        - 별도의 설정 X
    - queue
        - type과 동일하나, 이를 숫자로 표현한 방법
    - start
        - 시작 번호 : default 0 (처음)
    - count
        - 몇개의 match를 보여줄지 설정
        - default 20, 0 ~ 100사이의 값
        - count 만큼의 match를 가져다 준다.

- 만약 1월 1일부터 1월 31일까지경기를 200회 진행했다면,
    - 1640998800부터 1643673599까지의 데이터를
    - count를 20으로 두었을 때, start를 0,20,40,60,...,180으로 바꿔가며 수집해야 한다.

In [9]:
start_time = datetime.datetime(2022,1,1,0,0) # 1월 1일 0시 0분
start_time = int(time.mktime(start_time.timetuple()))
print(start_time)

1640962800


In [10]:
end_time = datetime.datetime(2022,1,31,23,59,59) # 1월 31일 23시 59분
end_time = int(time.mktime(end_time.timetuple()))
print(end_time)

1643641199


In [11]:
start = 0

In [12]:
count = 20

In [13]:
request_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{user_puuid}/ids?\
startTime={start_time}\
&endTime={end_time}\
&start={start}\
&count={count}'

In [14]:
response = requests.get(request_url, headers = headers)
result = response.json()
print('Status Code : ',response)

Status Code :  <Response [200]>


반복문으로 모든 경기 가져오기
    - count를 늘리고
    - start를 조정하자.

In [15]:
match_ids = []
start = 0
count = 50
while True : 
    request_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{user_puuid}/ids?\
startTime={start_time}\
&endTime={end_time}\
&start={start}\
&count={count}'
    
    response = requests.get(request_url, headers = headers)
    result = response.json()    
    match_ids.extend(result)
    match_ids = list(set(match_ids)) # 중복 제거
    
    if len(match_ids) < start : 
        print(f'1월 한달간 최종 {len(match_ids)}개의 경기 결과가 누적되었습니다.')
        break;
    else : 
        print(f'현재 총 {len(match_ids)}개의 경기 결과가 누적되었습니다.')
        start += count

현재 총 50개의 경기 결과가 누적되었습니다.
현재 총 100개의 경기 결과가 누적되었습니다.
현재 총 150개의 경기 결과가 누적되었습니다.
현재 총 168개의 경기 결과가 누적되었습니다.
1월 한달간 최종 168개의 경기 결과가 누적되었습니다.


## 여기까지하고 깨달은 사실..
- 매치 결과 api인 matches-v4가 현재 비활성 상태이다.
- https://developer.riotgames.com/api-status/
- 이렇게 된거 TFT 로 전향하겠습니다.
- 나중에 활성화 되면 테스트해보세요

## 4) 매치 정보 가져오기... ㅎㅎ

TFT는 최근 20게임만 지원한다.

- https://static.developer.riotgames.com/docs/lol/queues.json
- https://developer.riotgames.com/apis#tft-match-v1

In [16]:
request_url = f'https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{user_puuid}/ids'

In [17]:
print('https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/3y8bJywRuf70qBpin93Y9qUSt5DnqhTCwrnRbcJ1dKOqTD19UxYkWdkRMUdvr0aUS4gnib8HLCfmOA/ids?count=20')

https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/3y8bJywRuf70qBpin93Y9qUSt5DnqhTCwrnRbcJ1dKOqTD19UxYkWdkRMUdvr0aUS4gnib8HLCfmOA/ids?count=20


In [18]:
response = requests.get(request_url, headers = headers)
matches = response.json()
print('Status Code : ',response)
matches

Status Code :  <Response [200]>


['KR_5730084909',
 'KR_5729918927',
 'KR_5729874139',
 'KR_5729452302',
 'KR_5694091410',
 'KR_5660866876',
 'KR_5654286829',
 'KR_5654332517',
 'KR_5654251339',
 'KR_5654163747',
 'KR_5649772977',
 'KR_5649666465',
 'KR_5649626899',
 'KR_5649399594',
 'KR_5649430235',
 'KR_5647332040',
 'KR_5647156094',
 'KR_5647018730',
 'KR_5646981556',
 'KR_5646882348']

## 5) 경기 결과 가져오기

In [19]:
request_url = f'https://asia.api.riotgames.com/tft/match/v1/matches/{matches[0]}'
response = requests.get(request_url, headers = headers)
match_result = response.json()
print('Status Code : ',response)

Status Code :  <Response [200]>


- 결과 해석

- MatchDto
    - 게임 정보
    
- MetadataDto
    - data_version
    - match_id
    - 참여자 정보
- infoDto : 게임정보
    - 진행 날짜
    - 게임 길이
    - 게임 타입 (일반, 랭크 등)
    - 게임 버전
    - 참여자
    - queue_id
    - tft_set_number
- 이하 생략...

- 살펴볼 정보들
    - 유저에게 가한 데미지
    - 제거한 유저 수
    
    - 남은 골드
    - 올린 레벨
    
    - 살아남은 라운드
    - 최종 순위
    
    - 사용한 특성
    - 사용한 아이템
    - 사용한 유닛
- https://developer.riotgames.com/docs/tft 에서 item json 파일 다운

In [20]:
with open('items.json','r') as f : 
    items_list = json.load(f)
f.close()

items_dict = dict([[x.get('id'), x.get('name')] for x in items_list])

In [21]:
match_user_info = [x for x in match_result.get('info').get('participants') if x.get('puuid') == user_puuid][0]

In [22]:
total_damage_to_players = match_user_info.get('total_damage_to_players')
players_eliminated = match_user_info.get('players_eliminated')
gold_left = match_user_info.get('gold_left')
level = match_user_info.get('level')
last_round = match_user_info.get('last_round')
placement = match_user_info.get('placement')

In [23]:
match_user_traits =  match_user_info.get('traits')
match_user_units =  match_user_info.get('units')

In [24]:
match_traits = [[x.get('name'), x.get('num_units')]  for x in match_user_traits]
match_units = [[x.get('character_id'), x.get('tier')] for x in match_user_units]
match_items = [x.get('items') for x in match_user_units]

모든 경기에 대해서 살펴보기

In [25]:
total_damage_to_players = 0
players_eliminated = 0
gold_left = 0
level = 0
last_round = 0
placement = 0

match_traits = {}
match_units = {}
match_items = {}

for match in matches : 
    request_url = f'https://asia.api.riotgames.com/tft/match/v1/matches/{match}'
    response = requests.get(request_url, headers = headers)
    match_result = response.json()
    
    match_user_info = [x for x in match_result.get('info').get('participants') if x.get('puuid') == user_puuid][0]
    total_damage_to_players += match_user_info.get('total_damage_to_players')
    players_eliminated += match_user_info.get('players_eliminated')
    gold_left += match_user_info.get('gold_left')
    level += match_user_info.get('level')
    last_round += match_user_info.get('last_round')
    placement += match_user_info.get('placement')
    match_user_traits =  match_user_info.get('traits')
    match_user_units =  match_user_info.get('units')
    
    match_traits.update({trait : match_traits.get(trait,0)+unit_num for trait, unit_num in [[x.get('name'), x.get('num_units')] for x in match_user_traits]})
    match_units.update({character : match_units.get(character,0)+1 for character, tier in [[x.get('character_id'), x.get('tier')] for x in match_user_units]})
    for item in [item for items in [x.get('items') for x in match_user_units] for item in items] : 
        match_items[item] = match_items.get(item,0) + 1

In [26]:
print(f"""
한 판당 평균 가한 데미지       : {total_damage_to_players / 20}
한 판당 평균 제거 적의 수      : {players_eliminated / 20}
한 판당 평균 남긴 골드 량      : {gold_left / 20}
한 판당 평균 레벨              : {level / 20}
한 판당 평균 살아남은 라운드   : {last_round / 20}
한 판당 평균 순위              : {placement / 20}

20판동안 가장 많이 사용한 특성 : {[x[0] for x in sorted([[trait, num] for trait, num in match_traits.items()], key = lambda x: x[1], reverse= True)[:5]]},
20판동안 가장 많이 사용한 유닛 : {[x[0] for x in sorted([[unit, num] for unit, num in match_units.items()], key = lambda x: x[1], reverse= True)[:5]]},
20판동안 가장 많이 사용한 템   : {[items_dict.get(x[0]) for x in sorted([[item, num] for item, num in match_items.items()], key = lambda x: x[1], reverse= True)[:5]]},

""")


한 판당 평균 가한 데미지       : 147.9
한 판당 평균 제거 적의 수      : 1.5
한 판당 평균 남긴 골드 량      : 14.3
한 판당 평균 레벨              : 8.3
한 판당 평균 살아남은 라운드   : 35.1
한 판당 평균 순위              : 2.75

20판동안 가장 많이 사용한 특성 : ['Set6_Enchanter', 'Set6_Scholar', 'Set6_Scrap', 'Set6_Academy', 'Set6_Mutant'],
20판동안 가장 많이 사용한 유닛 : ['TFT6_Janna', 'TFT6_Taric', 'TFT6_Yuumi', 'TFT6_Orianna', 'TFT6_Jayce'],
20판동안 가장 많이 사용한 템   : ['Infinity Edge', 'Redemption', 'Bramble Vest', "Banshee's Claw", 'Ionic Spark'],




## 평균 순위 2.75... 그저 빛